In [144]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DateType
from pyspark.sql.functions import sum as spark_sum
import os
from pyspark.sql.functions import col, to_date, sum as spark_sum, concat_ws
from pyspark.sql.functions import sum as spark_sum
import time
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from IPython.display import clear_output

In [145]:
# Initialize SparkSession
def initialize_spark_session():
    """
    Initializes a SparkSession.

    Returns:
        SparkSession: Initialized SparkSession object.
    """
    return SparkSession.builder \
        .appName("Read Parquet Files") \
        .getOrCreate()

# Define schema for Parquet files
def define_schema():
    """
    Defines the schema for Parquet files.

    Returns:
        StructType: Defined schema for Parquet files.
    """
    return StructType([
        StructField("timestamp", TimestampType(), True),
        StructField("date", DateType(), True),
        StructField("year", IntegerType(), True),
        StructField("month", IntegerType(), True),
        StructField("day", IntegerType(), True),
        StructField("hour", IntegerType(), True),
        StructField("minute", IntegerType(), True),
        StructField("schema_name", StringType(), True),
        StructField("table_name", StringType(), True),
        StructField("insert", IntegerType(), True),
        StructField("update", IntegerType(), True),
        StructField("truncate", IntegerType(), True),
        StructField("delete", IntegerType(), True),
    ])

# Read Parquet files from given base path
def read_parquet_files(base_path):
    """
    Reads Parquet files from the given base path.

    Args:
        base_path (str): Base path where Parquet files are located.

    Returns:
        DataFrame: Combined DataFrame containing data from all Parquet files.
    """
    subfolders = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]
    combined_df = spark.createDataFrame([], schema)

    for subfolder in subfolders:
        file_path = os.path.join(base_path, subfolder)
        df = spark.read.schema(schema).parquet(file_path)
        combined_df = combined_df.union(df)
    
    return combined_df

# Group combined DataFrame by date, hour, schema_name, and table_name
def group_by_date_hour(combined_df):
    """
    Groups combined DataFrame by date, hour, schema_name, and table_name.

    Args:
        combined_df (DataFrame): Combined DataFrame containing data from Parquet files.

    Returns:
        DataFrame: Grouped DataFrame with aggregated operations counts.
    """
    grouped_by_hour = combined_df.groupBy('date','hour', 'schema_name', 'table_name') \
        .agg(spark_sum('insert').alias('insert_sum'),
             spark_sum('update').alias('update_sum'),
             spark_sum('truncate').alias('truncate_sum'),
             spark_sum('delete').alias('delete_sum'))

    grouped_with_ops = grouped_by_hour.withColumn('Total operations', 
                                                  col('insert_sum') + col('update_sum') + col('truncate_sum') + col('delete_sum'))
    grouped_with_ops = grouped_with_ops.select('date', 'hour', 'schema_name', 'table_name', 
                                               'insert_sum', 'update_sum', 'truncate_sum', 'delete_sum', 'Total operations')
    return grouped_with_ops.orderBy(grouped_with_ops['date'].desc(), grouped_with_ops['hour'].desc(),grouped_with_ops['Total operations'].desc())

# Get top 3 tables based on total operations
def get_top_3_tables(grouped_with_ops):
    """
    Gets the top 3 tables based on total operations.

    Args:
        grouped_with_ops (DataFrame): DataFrame with aggregated operations counts.

    Returns:
        DataFrame: DataFrame containing top 3 tables based on total operations.
    """
    window_spec = Window.partitionBy('date', 'hour').orderBy(col('Total operations').desc())
    ranked_df = grouped_with_ops.withColumn('rank', row_number().over(window_spec))
    return ranked_df.filter(col('rank') <= 3)


In [ ]:
# Define the base path where Parquet files are located
base_path = '/opt/rpx/repos/jupyter/notebook/parquet'

# Initialize SparkSession
spark = initialize_spark_session()

# Define schema for Parquet files
schema = define_schema()

# Read Parquet files from the base path and combine into a DataFrame
combined_df = read_parquet_files(base_path)

# Print the schema of the combined DataFrame
combined_df.printSchema()

# Group the combined DataFrame by date, hour, schema_name, and table_name and display the results
grouped_with_ops = group_by_date_hour(combined_df)
grouped_with_ops.show(1000)

# Print the count of rows and number of columns in the grouped DataFrame
print((grouped_with_ops.count(), len(grouped_with_ops.columns)))

# Calculate the sum of values in the "Total operations" column and print the result
column_sum = grouped_with_ops.select(spark_sum("Total operations")).collect()[0][0]
print("Sum of values in the column:", column_sum)


In [104]:
top3_df = get_top_3_tables(grouped_with_ops)
top3_df.show()


+----------+----+-----------+--------------------+----------+----------+------------+----------+----------------+----+
|      date|hour|schema_name|          table_name|insert_sum|update_sum|truncate_sum|delete_sum|Total operations|rank|
+----------+----+-----------+--------------------+----------+----------+------------+----------+----------------+----+
|2024-04-05|   7|        ldc|lit_docs_ptab_pet...|    640772|         0|           0|    640772|         1281544|   1|
|2024-04-05|   7|       docs|                docs|    615111|         0|           0|    615111|         1230222|   2|
|2024-04-05|   7|        ldc|lit_docs_patent_i...|    464625|         0|           0|    464625|          929250|   3|
+----------+----+-----------+--------------------+----------+----------+------------+----------+----------------+----+

